# Introduction
ASL is a prominent language in the American deaf community. With this in mind, I have a dataset of images containing the ASL dictionary, excluding the letters that need motion. The dataset was already flattened into greyscale values, ready for model input with a few preprocessing changes. This model will help translate ASL images to text for those who do not understand it, or want to learn it. 

# Analysis 
The first exploratory item I did was visualizing a few images from the dataset. Not only did I want to ensure the data accuracy, but I wanted to see what resolution I was working with. It was hard to do any summary statistics or heatmaps, etc. due to the data being flattened images so no EDA would have been useful in this case.

However, the images have a 28 x 28 x 1 size, a very low resolution image of hands depicting each letter of the american alphabet. Since the images have been flattened into columns of greyscale values, there will be 784 columns, and all values except for the predictor variable were scaled between 0-1. The predictor variable is a single number that denotes a letter of the alphabet, excluding J and Z which require motion. 


In [ ]:
#| label: Random Images from dataset
#| fig-cap: Random Images from Dataset
#| echo: false

import warnings
warnings.filterwarnings('ignore')
import yaml

import numpy as np
import pandas as pd

from keras.datasets import mnist
import keras as kb

from sklearn.preprocessing import LabelBinarizer

from plotnine import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score

train_df = pd.read_csv("/Users/kelseyhawkins/Desktop/CPSC_Courses/CPSC_393/sign_mnist_train.csv")
test_df = pd.read_csv("/Users/kelseyhawkins/Desktop/CPSC_Courses/CPSC_393/sign_mnist_test.csv")

# Separating X and Y
y_train = train_df['label']
y_test = test_df['label']

del train_df['label']
del test_df['label']

# rescale data to be 0-1 instead of 0-255
trainX = train_df.astype("float32") / 255.0
testX = test_df.astype("float32") / 255.0

# change the labels to be in the correct format
lb = LabelBinarizer()
trainY = lb.fit_transform(y_train)
testY = lb.transform(y_test)

trainX.head()
trainX.shape

print(trainX.shape,
trainY.shape)

print(testX.shape,
testY.shape)

# Visualize some images!!!
import matplotlib.pyplot as plt

# I used different names cuz i wanted to reshape them without
# Changing the original data put into the model :)
x_train = train_df.values
x_test = test_df.values
x_train_vis = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)
f, ax = plt.subplots(2,5)
f.set_size_inches(10, 10)
k = 0
for i in range(2):
    for j in range(5):
        ax[i,j].imshow(x_train_vis[k].reshape(28, 28) , cmap = "gray")
        k += 1
    #plt.tight_layout()    


# Methods
The first model I created is a Deep Feed-Forward Neural Network. I started with a simple model, built in class that had about 6 hidden layers. I then expanded it by adding more dropout, dense, and batch normalization layers. Each dense layer has an activation function of reLU, and a kernel regularizer of L2 on two early dense layers. The last dense layer outputs 24 nodes with a softmax activation function for predicted probabilities for each letter. When I compiled my model, I tested multiple optimizers including SGD, Adam, and RMSprop. The model included early stopping to prevent overfitting. 

The second model I created is a Random Forest Classifier. I created a simple one to get a general sense of the model outputs and comparison to the DNN. Then, I built a tuned RFC by utilizing the Randomized Search CV module. The parameters I tuned were the number of estimators, max depth, min samples split and leaf. The scorers were a combination of accuracy and F1 for a combination between all three classification metrics. 

# Results
Detailed discussion of how your model performed. Include a discussion about whether or not Deep Learning was necessary in this situation.


In [ ]:
#| label: Random Images from dataset
#| fig-cap: Random Images from Dataset
#| echo: false

import warnings
warnings.filterwarnings('ignore')
import yaml

import numpy as np
import pandas as pd

from keras.datasets import mnist
import keras as kb

from sklearn.preprocessing import LabelBinarizer

from plotnine import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score


# Reflection
In this assignment, I did not come across any struggles or problems with the code or analysis. However I did refer back to the math of the models to justify my answers and think about why the models performed the way they did. It was interesting to be able to quantify them with a simple classification problem and think about how each model handles class imbalances in the slightest way, by about 20%. In the future, I would approach it the same way as I did here, as it seemed to work well. 
